# Model for construct a conv net structure 

In [1]:
import tensorflow as tf
import scipy.io as sio
import numpy as np

In [2]:
IMAGE_HEIGHT = 56
IMAGE_WIDTH = 120
X = tf.placeholder(shape=[None,IMAGE_HEIGHT, IMAGE_WIDTH,1], dtype=tf.float32, name='X-input')
is_training = tf.placeholder(dtype=tf.bool)
sess = tf.InteractiveSession()

## inference

In [12]:
def inference(images_placeholder, is_training, depth1, depth2, depth3, dense1_units, dense2_units):

    training_mode = is_training is not None
    print('images_placeholder.shape:', images_placeholder.shape)

    # layer1:bn-conv-relu(depth1)-pool
    with tf.name_scope('conv1'):
        conv = tf.layers.conv2d(
            inputs=images_placeholder,
            filters=depth1,
            kernel_size=[3, 3],
            padding="same",
            activation=tf.nn.relu
        )
        print('conv1.shape:', conv.shape)

        pool = tf.layers.max_pooling2d(inputs=conv, pool_size=[2, 2], strides=2)
        print('pool1.shape:', pool.shape)


    # layer2:bn-conv-relu(depth2)-pool
    with tf.name_scope('conv2'):
        conv = tf.layers.conv2d(
            inputs=pool,
            filters=depth2,
            kernel_size=[3, 3],
            padding="same",
            activation=tf.nn.relu
        )
        print('conv2.shape:', conv.shape)
        
#         conv_without_pool = tf.layers.conv2d(
#             inputs=conv,
#             filters=depth2,
#             kernel_size=[3, 3],
#             padding="same",
#             activation=tf.nn.relu
#         )
#         print('conv2_without_pool.shape:', conv_without_pool.shape)

        pool = tf.layers.max_pooling2d(inputs=conv, pool_size=[2, 2], strides=2)
        print('pool2.shape:', pool.shape)

    with tf.name_scope('conv3'):
        conv = tf.layers.conv2d(
            inputs=pool,
            filters=depth3,
            kernel_size=[3, 3],
            padding="same",
            activation=tf.nn.relu
        )
        print('conv3.shape:', conv.shape)

#         conv_without_pool = tf.layers.conv2d(
#             inputs=conv,
#             filters=depth2,
#             kernel_size=[3, 3],
#             padding="same",
#             activation=tf.nn.relu
#         )
#         print('conv3_without_pool.shape:', conv_without_pool.shape)

        pool = tf.layers.max_pooling2d(inputs=conv, pool_size=[2, 2], strides=2)
        print('pool3.shape:', pool.shape)

## load data

In [15]:
def loadEEGData():
    dataPath = 'F:/Deep Learning/eeg/dataset/wym/'

    # 4050 每人162个图
    # 450 每人18个图
    X_train = sio.loadmat(dataPath+'train_x.mat')['train_x']
    X_test = sio.loadmat(dataPath+'test_x.mat')['test_x']
    y_train = sio.loadmat(dataPath+'train_y.mat')['train_y']
    y_test = sio.loadmat(dataPath+'test_y.mat')['test_y']

    # 此处label不用one-hot形式表示，而是用对应的类别直接表示（0~24）
    y_train = np.argmax(y_train, axis=1)
    y_test = np.argmax(y_test, axis=1)

    X_train = np.reshape(X_train, (-1, 56, 120, 1))
    X_test = np.reshape(X_test, (-1, 56, 120, 1))

    mean_image = np.mean(X_train, axis=0)
    X_train -= mean_image
    X_test -= mean_image

    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = loadEEGData()
print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)
print('X_test.shape: ', X_test.shape)
print('y_test.shape: ', y_test.shape)

X_train.shape:  (4050, 56, 120, 1)
y_train.shape:  (4050,)
X_test.shape:  (450, 56, 120, 1)
y_test.shape:  (450,)


## test model structure

In [16]:
inference(X, True, 32, 64, 128, 1, 1)

images_placeholder.shape: (?, 56, 120, 1)
conv1.shape: (?, 56, 120, 32)
pool1.shape: (?, 28, 60, 32)
conv2.shape: (?, 28, 60, 64)
pool2.shape: (?, 14, 30, 64)
conv3.shape: (?, 14, 30, 128)
pool3.shape: (?, 7, 15, 128)
